아래는 ALO 기본 설정 및 라이브러리 설치 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 하고, 지속적으로 문제가 있을 시 문의바랍니다

In [1]:
import argparse
import time
import os


original_directory = os.getcwd()

# os.chdir(os.path.abspath(os.path.join('..')))
os.chdir("./alo")
from src.alo import ALO

alo = ALO()

alo.preset()

pipelines = list(alo.asset_source.keys())

from src.external import external_load_data, external_save_artifacts

pipeline = pipelines[0]

external_load_data(pipeline, alo.external_path, alo.external_path_permission)

alo.install_steps(pipeline, alo.control["get_asset_source"])
asset_config = alo.asset_source[pipeline]

======================================== Start dependency installation : << master >> 
>> Start checking existence & installing package - alolib@git+http://mod.lge.com/hub/dxadvtech/aicontents-framework/alolib-source.git@release-1.1 | Progress: ( 1 / 1 total packages )
- << alolib@git+http://mod.lge.com/hub/dxadvtech/aicontents-framework/alolib-source.git@release-1.1 >> already exists
======================================== Start dependency installation : << force-reinstall >> 
======================================== Finish dependency installation 

STTIME does not exist. instead get current time
>> << /home/jovyan/ic/alo/assets/ >> directory already exists.
>> Start removing pre-existing input data before fetching external data: train
>> Start loading external << /nas001/users/ruci.sung/mnist/train/ >> data into << input >> directory.
>> s3 private key file << load_s3_key_path >> loaded successfully.
>> [@ train_pipeline] Start fetching external data from << /nas001/users/ruci.sung/

In [2]:
import pandas as pd

def make_ground_truth(path,save_path,data_type='png'):

    path_list = []
    col_names = ['label','image_path']
    
    data_type_dict = {}
    data_type_dict['png'] = ['png']
    data_type_dict['jpg'] = ['jpg']
    data_type_dict['both'] =  ['png','jpg']
    
    
    WALK = os.walk(path)
    i = 0
    for (Root,Dir,fles) in WALK:
        for fle in fles:
            if fle.split('.')[-1] in data_type_dict[data_type]:
                path_list.append([Root.split('/')[-1] , os.path.join(Root,fle)])
                if i<5:
                    print(path_list[-1])
                    i += 1
            
    df = pd.DataFrame(path_list,columns = col_names)
    df.to_csv(save_path,index=False)
    
    return path_list,df


## Ground truth data 만들기 샘플 코드

In [3]:
# train 혹은 test 데이터 각각에 대해 ground truth 파일을 각각 만들어야 합니다.

# train 혹은 test 데이터가 있는 폴더명을 입력합니다.
dataDir = '/nas001/users/yoonji.suh/sample_data/mnist_png/mnist_png/train/'
# ground truth 파일이 저장될 경로를 확장자 까지 입력합니다.
saveDir = '/home/jovyan/testCsv.csv'

lst,df = make_ground_truth(dataDir,saveDir)
print('complete')
df.head()

['5', '/nas001/users/yoonji.suh/sample_data/mnist_png/mnist_png/train/5/100.png']
['5', '/nas001/users/yoonji.suh/sample_data/mnist_png/mnist_png/train/5/42085.png']
['5', '/nas001/users/yoonji.suh/sample_data/mnist_png/mnist_png/train/5/4119.png']
['5', '/nas001/users/yoonji.suh/sample_data/mnist_png/mnist_png/train/5/7260.png']
['5', '/nas001/users/yoonji.suh/sample_data/mnist_png/mnist_png/train/5/52231.png']
complete


,label,image_path
0,5,/nas001/users/yoonji.suh/sample_data/mnist_png...
1,5,/nas001/users/yoonji.suh/sample_data/mnist_png...
2,5,/nas001/users/yoonji.suh/sample_data/mnist_png...
3,5,/nas001/users/yoonji.suh/sample_data/mnist_png...
4,5,/nas001/users/yoonji.suh/sample_data/mnist_png...


## Train workflow 
### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- 필요한경우 input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [4]:
# IC train asset 순서에 따라 step 순서를 입력합니다. (input(0) - train(1))
step = 0
input_args = alo.user_parameters[pipeline][step]

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 
# input_args['path_column'] = ['path']
input_args

{'step': 'input',
 'args': [{'input_path': 'train/',
   'y_column': 'label',
   'path_column': 'image_path',
   'data_source_type': 'batch',
   'label_names': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
   'prediction_column_prefix': 'pred',
   'probability_column_prefix': 'prob',
   'set_images': True}]}

##### Input asset 실행 

In [5]:
data = pipe_val = {} # 초기 input asset process 세팅

data_input, config_input = alo.process_asset_step(asset_config[step], step, pipeline)
# data_input: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_input['dataframe']으로 확인할 수 있습니다. 
data_input['dataframe'].head(10) 

************************************************************
************************************************************
>>>>> Load path : ['/home/jovyan/ic/alo//input/train//']
/home/jovyan/ic/alo//.asset_interface/train_pipeline/ 폴더와 하위 디렉토리가 삭제되었습니다.
/home/jovyan/ic/alo//.asset_interface/train_pipeline/ 폴더가 생성되었습니다.


/home/jovyan/ic/alo/assets/input/data_input.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['new_image_path'] = data.apply(save_image_as_npy, axis=1)
/home/jovyan/ic/alo/assets/input/data_input.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.path_column] = data['new_image_path']
/home/jovyan/ic/alo/assets/input/data_input.py:271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

,image_path,label
0,/home/jovyan/ic/alo//.asset_interface/train_pi...,7
1,/home/jovyan/ic/alo//.asset_interface/train_pi...,7
2,/home/jovyan/ic/alo//.asset_interface/train_pi...,7
3,/home/jovyan/ic/alo//.asset_interface/train_pi...,8
4,/home/jovyan/ic/alo//.asset_interface/train_pi...,4
5,/home/jovyan/ic/alo//.asset_interface/train_pi...,3
6,/home/jovyan/ic/alo//.asset_interface/train_pi...,3
7,/home/jovyan/ic/alo//.asset_interface/train_pi...,1
8,/home/jovyan/ic/alo//.asset_interface/train_pi...,6
9,/home/jovyan/ic/alo//.asset_interface/train_pi...,1


</br>

### 2. train asset 
##### train asset의 args수정 및 확인
- 필요한경우 IC_args의 항목을 ***IC_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [6]:
# IC train asset 순서에 따라 step 순서를 입력합니다. (input(0) - train(1))
step = 1
ic_args = alo.user_parameters[pipeline][step]

# 아래 주석을 풀어 ic_args를 원하는 값으로 수정합니다. 
# ic_args['epochs'] = 100
ic_args

{'step': 'train',
 'args': [{'model_type': 'mobilenetv1',
   'input_shape': [28, 28, 1],
   'resize_shape': [224, 224, 3],
   'num_classes': 10,
   'epochs': 10,
   'batch_size': 64,
   'train_ratio': 0.8,
   'rand_augmentation': False}]}

##### train asset 실행 

In [ ]:
data = pipe_val = {} # 초기 input asset process 세팅

data_train, config_train = alo.process_asset_step(asset_config[step], step, pipeline, data_input, config_input)
# data_train: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_train: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_train['dataframe']으로 확인할 수 있습니다. 
data_train['dataframe'].head(10) 

************************************************************
************************************************************
>> Successfully got model path for saving or loading your AI model: 
 /home/jovyan/ic/alo//.train_artifacts/models/train/
>> Successfully got << output path >> for saving your data into csv or jpg file: 
 /home/jovyan/ic/alo//.train_artifacts/output/train/ 
 L [NOTE] The names of output file must be fixed as << output.csv, output.jpg >> 


2023-10-26 01:29:49.105702: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jovyan/conda/envs/ic/lib/python3.10/site-packages/cv2/../../lib64:/home/jovyan/conda/envs/py39_gpu/lib/:/home/jovyan/conda/lib/:/home/jovyan/conda/envs/aic_gpu/lib/:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-26 01:29:49.105753: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-26 01:29:49.105782: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-yoonji-2esuh): /proc/driver/nvidia/version does not exist
2023-10-26 01:29:49.113380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Lib

# of original training data:800 / # of augmented training data:0 / # of validation data:200
resize_shape: (224, 224, 3)
selected model: mobilenet V1
Epoch 1/10
      8/Unknown - 19s 2s/step - loss: 1.6235 - accuracy: 0.5371

<br>

## Inference workflow 
### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- 필요한경우 input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
pipeline = pipelines[1]
external_load_data(pipeline, alo.external_path, alo.external_path_permission)

alo.install_steps(pipeline, alo.control["get_asset_source"])

In [ ]:
# IC inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - inference(1))
step = 0
input_args = alo.user_parameters[pipeline][step]

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 
# input_args['path_column'] = ['path']
input_args

##### Input asset 실행 

In [ ]:
data = pipe_val = {} # 초기 input asset process 세팅

data_input, config_input = alo.process_asset_step(asset_config[step], step, pipeline)
# data_input: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_input['dataframe']으로 확인할 수 있습니다. 
data_input['dataframe'].head(10) 

<br>

### 2. inference asset 
##### inference asset의 args수정 및 확인
- 필요한경우 IC_args의 항목을 ***IC_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# IC inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - inference(1))
step = 1
ic_args = alo.user_parameters[pipeline][step]

# 아래 주석을 풀어 ic_args를 원하는 값으로 수정합니다. 
# ic_args['epochs'] = 100
ic_args

##### inference asset 실행 

In [ ]:
data = pipe_val = {} # 초기 input asset process 세팅

data_input, config_input = alo.process_asset_step(asset_config[step], step, pipeline, data_input, config_input)
# data_input: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_input['dataframe']으로 확인할 수 있습니다. 
data_input['dataframe'].head(10) 